In [1]:
!pip install peft
!pip install Faiss-cpu
!pip install langchain


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.8 MB/s eta 0:00:00


In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00


In [1]:

from peft import PeftModel, PeftConfig
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain.llms import OpenAIChat
from langchain.chat_models import ChatOpenAI

import torch
import re
import base64
import os

In [2]:
SEARCH_NUM = 3
conversation_pairs = []
loaded_memory =""
dialogues = ""
query2=""

In [3]:
from langchain.embeddings import HuggingFaceEmbeddings

In [6]:
pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=6946f1de257906355b290a588da9844c3eddfd980136b93dbe82c513e187b039
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [4]:
# csv load
from langchain.document_loaders import TextLoader

loader = TextLoader("/content/ex1.txt")
data=loader.load()

# text split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=0)
data_split = text_splitter.split_documents(data)

# embedding
embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")
db = FAISS.from_documents(data_split, embeddings)

docs_memory = []
cnt=0
from transformers import StoppingCriteria, StoppingCriteriaList

class StoppingCriteriaSub(StoppingCriteria):
    def __init__(self, stops = [], encounters=1):
        super().__init__()
        self.stops = [stop for stop in stops]

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        for stop in self.stops:
            if torch.all((stop == input_ids[0][-len(그만):])).item():
                return True

        return False

In [65]:
# prompt template
template = '''
    너는 사실만을 말하는 금융 전문가야.
    질문에 문서에 기반해서 답을 하고, 문서와 관련이 없거나 모호하다면 내용을 찾을 수 없습니다. 라고 대답해.
    문서에서 질문에 대한 근거를 찾을 수 없거나 모호하다면 내용을 찾을 수 없습니다. 라고 답해.
    문서 = {docs_input}
    질문 = {query_input}
'''


In [66]:
# prompt
prompt = PromptTemplate(
    input_variables=[
        "docs_input",
        "query_input",
    ],
    template=template,)

In [59]:
# search similarity
def db_search(query: str, k: int):
    docs = db.similarity_search(query, k)
    return docs


In [60]:
prompt

PromptTemplate(input_variables=['docs_input', 'query_input'], template='\n    너는 사실만을 말하는 금융 전문가야.\n    질문에 문서에 기반해서 답을 하고, 문서와 관련이 없거나 모호하다면 내용을 찾을 수 없습니다. 라고 대답해.\n    문서에서 질문에 대한 근거를 찾을 수 없거나 모호하다면 내용을 찾을 수 없습니다. 라고 답해.\n    문서 = {docs_input}\n    질문 = {query_input}\n')

In [61]:
# chain_type_kwargs = {"prompt": prompt}

In [62]:
from langchain.chains import RetrievalQA

In [ ]:
# def process_llm_response(llm_response):
#     print(wrap_text_preserve_newlines(llm_response['result']))
#     print('\n\nSources:')

In [23]:
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
from transformers import TextStreamer, GenerationConfig

def load_local_llm():
  model='davidkim205/komt-mistral-7b-v1'
  peft_model_name = 'davidkim205/komt-mistral-7b-v1-lora'
  config = PeftConfig.from_pretrained(peft_model_name)
  bnb_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.bfloat16
  )
  config.base_model_name_or_path =model
  model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, quantization_config=bnb_config, device_map="auto")
  model = PeftModel.from_pretrained(model, peft_model_name)
  return model

In [17]:
!pip install sentencepiece

In [9]:
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
streamer = TextStreamer(tokenizer)

In [24]:
# llm = load_local_llm()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [41]:
def load_chain(_prompt) :
    chain = LLMChain(
        llm=llm,
        prompt=prompt
    )
    return chain

In [51]:
retriever = db.as_retriever(search_kwargs={"k": 3})

ValidationError: ignored

In [42]:
#run chain
def run_chain(prompt, docs,query: str):
    chain = load_chain(prompt)
    response = chain.run(
    docs_input=docs,
    query_input=query,
    )
    return response

In [43]:
query='삼성전자의 3분기 영업이익은?'

In [44]:
query='삼성전자의 3분기 영업이익은?'
docs = db_search(query, SEARCH_NUM)
docs = [doc.page_content for doc in docs]

In [45]:
docs

['삼성전자는 31일 “올 3분기(7~9월) 매출 67조4047원, 영업이익 2조4336억원을 기록했다”고 확정 공시했다. 매출과 영업이익은 지난해 동기 대비 각각 12%, 78%',
 '특히 삼성전자는 이날 3분기 DS(반도체) 부문에서 영업손실 3조7500억원을 기록했다고 밝혔다. DS부문 적자 규모가 올 1분기(4조 5800억원),',
 '삼성전자가 올 3분기 반도체 사업에서 3조7500억원의 적자를 기록했다. 직전분기(4조3600억원)에 비해 적자 폭을 약 6000억원 가량 줄였다.']

In [46]:
# full example

llm_response = qa_chain(query)


ValidationError: ignored

In [68]:
def gen(x,docs_input):
    generation_config = GenerationConfig(
        temperature=1,
        top_p=0.8,
        top_k=100,
        max_new_tokens=1024,
        early_stopping=True,
        do_sample=True,
    )
    template = f"""[INST]
        너는 사실만을 말하는 금융 전문가야.
        질문에 문서에 기반해서 답을 하고, 문서와 관련이 없거나 모호하다면 내용을 찾을 수 없습니다. 라고 대답해.
        문서 = {docs_input}
        질문 = {x} [/INST]
    """

    q = template
    # q = f"[INST]{x} [/INST]"
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        generation_config=generation_config,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        streamer=streamer,
    )
    result_str = tokenizer.decode(gened[0])

    start_tag = f"[/INST]"
    start_index = result_str.find(start_tag)

    if start_index != -1:
        result_str = result_str[start_index + len(start_tag):].strip()
    return result_str

# result = gen('제주도를 1박2일로 혼자 여행하려고 하는데 여행 코스를 만들어줘')

# print('##########')
# print(result)

In [79]:
query='삼성전자는 3분기에 반도체 사업에서 얼마나 적자를 기록했나요?'
docs = db_search(query, SEARCH_NUM)
docs = [doc.page_content for doc in docs]
gen(query,docs)[-1]

<s> [INST]
        너는 사실만을 말하는 금융 전문가야.
        질문에 문서에 기반해서 답을 하고, 문서와 관련이 없거나 모호하다면 내용을 찾을 수 없습니다. 라고 대답해.
        문서 = ['특히 삼성전자는 이날 3분기 DS(반도체) 부문에서 영업손실 3조7500억원을 기록했다고 밝혔다. DS부문 적자 규모가 올 1분기(4조 5800억원),', '삼성전자가 올 3분기 반도체 사업에서 3조7500억원의 적자를 기록했다. 직전분기(4조3600억원)에 비해 적자 폭을 약 6000억원 가량 줄였다.', '삼성전자는 31일 “올 3분기(7~9월) 매출 67조4047원, 영업이익 2조4336억원을 기록했다”고 확정 공시했다. 매출과 영업이익은 지난해 동기 대비 각각 12%, 78%']
        질문 = 삼성전자는 3분기에 반도체 사업에서 얼마나 적자를 기록했나요? [/INST]
    삼성전자는 3분기에 반도체 사업에서 3조7500억원의 적자를 기록했습니다.</s>


'>'

In [80]:
gen(query,docs)[-1]

<s> [INST]
        너는 사실만을 말하는 금융 전문가야.
        질문에 문서에 기반해서 답을 하고, 문서와 관련이 없거나 모호하다면 내용을 찾을 수 없습니다. 라고 대답해.
        문서 = ['특히 삼성전자는 이날 3분기 DS(반도체) 부문에서 영업손실 3조7500억원을 기록했다고 밝혔다. DS부문 적자 규모가 올 1분기(4조 5800억원),', '삼성전자가 올 3분기 반도체 사업에서 3조7500억원의 적자를 기록했다. 직전분기(4조3600억원)에 비해 적자 폭을 약 6000억원 가량 줄였다.', '삼성전자는 31일 “올 3분기(7~9월) 매출 67조4047원, 영업이익 2조4336억원을 기록했다”고 확정 공시했다. 매출과 영업이익은 지난해 동기 대비 각각 12%, 78%']
        질문 = 삼성전자는 3분기에 반도체 사업에서 얼마나 적자를 기록했나요? [/INST]
    삼성전자는 3분기에 반도체 사업에서 3조7500억원의 적자를 기록했다.</s>


'>'

In [83]:
query='삼성전자는 4분기에 어떤 성과를 기대하고 있나요?'
docs = db_search(query, SEARCH_NUM)
docs = [doc.page_content for doc in docs]
gen(query,docs)[-1]

<s> [INST]
        너는 사실만을 말하는 금융 전문가야.
        질문에 문서에 기반해서 답을 하고, 문서와 관련이 없거나 모호하다면 내용을 찾을 수 없습니다. 라고 대답해.
        문서 = ['4분기에는 반등을 노린다. 삼성전자 측은 “4분기는 글로벌 IT 수요가 점진적으로 개선될 것으로 전망되고 있다”며 “특히 DS부문은 HBM 등 고부가 제품 판매 확대 및 기술', '삼성전자는 31일 “올 3분기(7~9월) 매출 67조4047원, 영업이익 2조4336억원을 기록했다”고 확정 공시했다. 매출과 영업이익은 지난해 동기 대비 각각 12%, 78%', '삼성전자는 “메모리 분야에서 DDR5, 고대역폭 메모리(HBM) 등 고부가 제품의 판매 확대와 판매가 상승으로 직전분기 대비 적자폭이 축소됐다”며 “업황이 저점을 지나고 있다는']
        질문 = 삼성전자는 4분기에 어떤 성과를 기대하고 있나요? [/INST]
    삼성전자는 4분기에 글로벌 IT 수요가 점진적으로 개선될 것으로 전망하고 있습니다. 특히 DS부문은 HBM 등 고부가 제품 판매 확대 및 기술 개발에 집중하여 성과를 기대하고 있습니다.</s>


'>'

In [88]:
SEARCH_NUM=7

In [90]:
query=' 삼성의 DX 부문이 프리미엄 전략을 강화한다는데, 이로 인해 어떤 결과를 예상하고 있나요?'
docs = db_search(query, SEARCH_NUM)
docs = [doc.page_content for doc in docs]
gen(query,docs)[-1]

<s> [INST]
        너는 사실만을 말하는 금융 전문가야.
        질문에 문서에 기반해서 답을 하고, 문서와 관련이 없거나 모호하다면 내용을 찾을 수 없습니다. 라고 대답해.
        문서 = ['리더십에 집중하고 디스플레이와 DX부문은 프리미엄 전략을 강화해 견조한 수익성을 유지해나가겠다”고 밝혔다.', '4분기에는 반등을 노린다. 삼성전자 측은 “4분기는 글로벌 IT 수요가 점진적으로 개선될 것으로 전망되고 있다”며 “특히 DS부문은 HBM 등 고부가 제품 판매 확대 및 기술', '삼성전자는 “메모리 분야에서 DDR5, 고대역폭 메모리(HBM) 등 고부가 제품의 판매 확대와 판매가 상승으로 직전분기 대비 적자폭이 축소됐다”며 “업황이 저점을 지나고 있다는', '파운드리에서도 실적 부진은 지속됐지만 고성능컴퓨팅(HPC) 중심으로 역대 최대 분기 수주를 달성했다고 삼성은 밝혔다.', '등 부진에 허덕였지만 신작 폴더블폰 출시와 디스플레이 수요 증가가 호조로 작용했다.', '삼성전자는 31일 “올 3분기(7~9월) 매출 67조4047원, 영업이익 2조4336억원을 기록했다”고 확정 공시했다. 매출과 영업이익은 지난해 동기 대비 각각 12%, 78%', '특히 삼성전자는 이날 3분기 DS(반도체) 부문에서 영업손실 3조7500억원을 기록했다고 밝혔다. DS부문 적자 규모가 올 1분기(4조 5800억원),', '인식이 확산되며 부품 재고를 확보하기 위한 고객사의 구매 문의가 다수 접수됐다”고 밝혔다. 지난 4월 감산에 돌입하면서 메모리 제품 가격 하락 폭이 둔화하는 등 감산 동참 효과가']
        질문 =  삼성의 DX 부문이 프리미엄 전략을 강화한다는데, 이로 인해 어떤 결과를 예상하고 있나요? [/INST]
    삼성의 DX 부문이 프리미엄 전략을 강화한다는 것은 견조한 수익성을 유지해나가겠다는 의미입니다. 이는 삼성전자가 프리미엄 제품을 더 중점으로 개발하고 판매하여 수익성을 높이는 전략을 의미합니다. 따라서 이로 인해 삼성전

'>'

In [93]:
query='삼성전자가 반도체 사업에서 발생한 3분기 적자의 주요 원인은 무엇이었으며, 어떻게 개선되었나요?'
docs = db_search(query, SEARCH_NUM)
docs = [doc.page_content for doc in docs]
gen(query,docs)[-1]

<s> [INST]
        너는 사실만을 말하는 금융 전문가야.
        질문에 문서에 기반해서 답을 하고, 문서와 관련이 없거나 모호하다면 내용을 찾을 수 없습니다. 라고 대답해.
        문서 = ['특히 삼성전자는 이날 3분기 DS(반도체) 부문에서 영업손실 3조7500억원을 기록했다고 밝혔다. DS부문 적자 규모가 올 1분기(4조 5800억원),', '삼성전자가 올 3분기 반도체 사업에서 3조7500억원의 적자를 기록했다. 직전분기(4조3600억원)에 비해 적자 폭을 약 6000억원 가량 줄였다.', '삼성전자는 “메모리 분야에서 DDR5, 고대역폭 메모리(HBM) 등 고부가 제품의 판매 확대와 판매가 상승으로 직전분기 대비 적자폭이 축소됐다”며 “업황이 저점을 지나고 있다는', '삼성전자는 31일 “올 3분기(7~9월) 매출 67조4047원, 영업이익 2조4336억원을 기록했다”고 확정 공시했다. 매출과 영업이익은 지난해 동기 대비 각각 12%, 78%', '3분기 전체 실적에선 영업이익을 2조4336억원을 기록하면서 올 들어 처음으로 조(兆) 단위 영업이익을 기록했다. 상반기에 주력인 반도체 부문에서만 9조원이 넘는 적자를 기록하는', '파운드리에서도 실적 부진은 지속됐지만 고성능컴퓨팅(HPC) 중심으로 역대 최대 분기 수주를 달성했다고 삼성은 밝혔다.', '4분기에는 반등을 노린다. 삼성전자 측은 “4분기는 글로벌 IT 수요가 점진적으로 개선될 것으로 전망되고 있다”며 “특히 DS부문은 HBM 등 고부가 제품 판매 확대 및 기술', '2분기(4조3600억원)보다는 대폭 개선됐다.']
        질문 = 삼성전자가 반도체 사업에서 발생한 3분기 적자의 주요 원인은 무엇이었으며, 어떻게 개선되었나요? [/INST]
    삼성전자가 반도체 사업에서 발생한 3분기 적자의 주요 원인은 메모리 분야에서의 DDR5, 고대역폭 메모리(HBM) 등 고부가 제품의 판매 확대와 판매가 상승으로 인한 것으로 알려져 있습니다. 이러한 원인으로 인해

'>'

In [92]:
query='LG전자의 영업이익은 어떤가요?'
docs = db_search(query, SEARCH_NUM)
docs = [doc.page_content for doc in docs]
gen(query,docs)[-1]

기대하는<s> [INST]
        너는 사실만을 말하는 금융 전문가야.
        질문에 문서에 기반해서 답을 하고, 문서와 관련이 없거나 모호하다면 내용을 찾을 수 없습니다. 라고 대답해.
        문서 = ['삼성전자는 31일 “올 3분기(7~9월) 매출 67조4047원, 영업이익 2조4336억원을 기록했다”고 확정 공시했다. 매출과 영업이익은 지난해 동기 대비 각각 12%, 78%', '삼성전자는 “메모리 분야에서 DDR5, 고대역폭 메모리(HBM) 등 고부가 제품의 판매 확대와 판매가 상승으로 직전분기 대비 적자폭이 축소됐다”며 “업황이 저점을 지나고 있다는', '3분기 전체 실적에선 영업이익을 2조4336억원을 기록하면서 올 들어 처음으로 조(兆) 단위 영업이익을 기록했다. 상반기에 주력인 반도체 부문에서만 9조원이 넘는 적자를 기록하는', '4분기에는 반등을 노린다. 삼성전자 측은 “4분기는 글로벌 IT 수요가 점진적으로 개선될 것으로 전망되고 있다”며 “특히 DS부문은 HBM 등 고부가 제품 판매 확대 및 기술', '등 부진에 허덕였지만 신작 폴더블폰 출시와 디스플레이 수요 증가가 호조로 작용했다.', '파운드리에서도 실적 부진은 지속됐지만 고성능컴퓨팅(HPC) 중심으로 역대 최대 분기 수주를 달성했다고 삼성은 밝혔다.', '특히 삼성전자는 이날 3분기 DS(반도체) 부문에서 영업손실 3조7500억원을 기록했다고 밝혔다. DS부문 적자 규모가 올 1분기(4조 5800억원),', '삼성전자가 올 3분기 반도체 사업에서 3조7500억원의 적자를 기록했다. 직전분기(4조3600억원)에 비해 적자 폭을 약 6000억원 가량 줄였다.']
        질문 = LG전자의 영업이익은 어떤가요? [/INST]
    죄송하지만, 제가 주어진 문서와 관련이 없거나 모호한 질문에 대한 답변을 할 수 없습니다. 제가 도움을 드릴 수 있는 주제에 관한 질문이 있으시면 제가 도움을 드릴 수 있습니다.</s>


'>'